Notebook zu Python: Wissenschaftliches Rechnen: Korrelationen und Curve Fitting

Version 1.2, 12. März 2024, Informatik, EAH Jena

(c) Christina B. Class

In diesem Notebook verwenden wir `numpy` und `matplotlib.pyplot`. Daher sind die folgenden zwei `import` Statements wesentlich.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Des Weiteren verwenden wir noch die folgenden Module:
- `scipy.stats` für den Pearson's Product Moment Coefficient 
- `scipy.optimize` für die Funktion `curve_fit()`
- `sklearn.metrics` für das Bestimmtheitsmaß  $R^2$
- `np.polynomial.polynomial` die Funktionen `polyfit()` und `polyval()`


In [ ]:
import scipy.optimize as opt
import sklearn.metrics as masse 
import scipy.stats as stats
import numpy.polynomial.polynomial as poly

Im folgenden importieren wir das Modul `random`, mit dem Zufallszahlen erzeugt werden. Wir initializieren den Zustand mit 42, so dass wir auch bei Wiederholung des Notebooks, sofern wir alle Zellen in der gleichen Reihenfolge ausführen, die gleichen Werte erzeugen.

In [ ]:
import random as rnd
rnd.seed(42)

# 0. Erzeugung von Daten

In diesem Notebook werden wir mit künstlich erzeugten Daten arbeiten. 

Hierzu erzeugen wir zuerst die $x$-Werte und berechnen die $y$-Werte basierend auf einer vorgegebenen Funktion. 

Dann erzeugen wir ein gewisses Grundrauschen, Hierzu ersetzen wir die $y$-Werte durch einen Zufallswert basierend auf der Normalverteilung (<a href="https://de.wikipedia.org/wiki/Normalverteilung">Normalverteilung</a>) mit dem Erwartungswert $y$ und einer Standardabweichung. Diese können wir z.B. von der maximalen Differenz der $y$-Werte abhängig machen. Die `gauss()` Funktion steht im Modul `random`.

Eine erste Funktion erzeugt lineare Daten. Sie basiert auf der Gleichung linearer Funktionen: $f(x)=a\cdot x+b$. Wir definieren ein Intervall für die $x$-Werte und definieren den Standardabweichung für das Grundrauschen als einen Teil der Intervalllänge.  

In [ ]:
# div definiert den Wert, durch den die Länge des Intervalls für x-Werte definiert wird, um die
# Standardabweichung zu berechnen
def lineareDaten(a,b,div,xmin,xmax):
    x=np.linspace(xmin,xmax,100)
    y=a*x+b
    
    mu=y.mean()
    std=(y.max()-y.min())/div 
    noise=[rnd.gauss(y[i],std) for i in range(100)]
    y=y+np.array(noise)
    data=np.array([x,y]).transpose()
    return data

Für ein zweites Beispiel erzeugen wir eine verrauschte Kubikfunktion für die Gleichung $f(x)=a \cdot x^3+b \cdot x + c$:

In [ ]:
# div definiert den Wert, durch den die Länge des Intervalls für x-Werte definiert wird, um die
# Standardabweichung zu berechnen
def kubikDaten(a,b,c,div,xmin,xmax):
    x=np.linspace(xmin,xmax,100)
    y=a*x**3+b*x+c
    
    mu=y.mean()
    std=(y.max()-y.min())/div 
    noise=[rnd.gauss(y[i],std) for i in range(100)]
    y=y+np.array(noise)
    data=np.array([x,y]).transpose()
    return data

# 1. Curve-Fitting der linearen Funktion

## 1.1 Kennenlernen der Daten

Als Beispieldaten erzeugen wir nun Daten mit der Funktion `lineareDaten()` im Intervall $[-5,5]$ für die Funktion $f(x)=3.213 \cdot x +12$. Für die Berechnung der Standardabweichung dividieren wir durch 10. 

In [ ]:
data=lineareDaten(a=3.213,b=12,div=10,xmin=-5,xmax=5)

Wir gehen jetzt einfach mal davon aus, dass in `data` Daten stehen, die wir näher kennenlernen wollen. Hierzu machen wir zuerst einmal einen Plot:

In [ ]:
plt.plot(data[:,0],data[:,1],'.')

**Hinweis**: Da die Daten mit einem zufälligen Rauschen erzeugt wurden, sehen die Plots und Werte hier natürlich bei jedem Durchlauf etwas anders aus.

Wir haben nun die Vermutung, dass die Daten korreliert sind. Hierzu können wir den **Pearson's Product Moment Coefficient** berechnen.
Die Funktion `pearsonr()` erhält die X und die Y Werte und gibt zwei Werte zurück. Der erste Wert ist der Koeffizient, der zweite ist der $p$ Wert, der dabei hilft, den exakten Korrelationskoeffizienten eingrenzen zu können. (Gemessene Werte bilden die Wirklichkeit ja eigentlich nie ganz genau ab, insofern können auch solche berechneten Werte nicht genauer sein.)  

In [ ]:
pc,p=stats.pearsonr(data[:,0],data[:,1])
print("Der Korrelationskoeffizient ist {:.4f}.".format(pc))

Der Wert ist nahe bei eins, die beiden Werte korrelieren stark. Das sehen wir ja schon am Plot.
Nun wollen wir das Curve Fitting durchführen.

## 1.2 `curve_fit()`

Die Funktion `curve_fit()` im Modul `scipy.optimize` ist sehr flexibel. Sie geben die Funktionsgleichung an, deren Koeffizienten durch die Funktion bestimmt werden sollen. Dadurch können Sie eine Vielzahl von Funktionen angeben und sind nicht auf Polynome beschränkt. Viele Parameter erlauben verschiedene Anpassungen, u.a. können Sie auch das verwendete Verfahren auswählen. 

Um `curve_fit()` verwenden zu können, müssen wir eine  Funktion übergeben, für die die Koeffizienten bestimmt werden sollen. Als erster Parameter erhält diese Funktion `x`, dann die einzelnen Koeffizienten. Wie definieren eine Python Funktion für die Funktion $f(x)=a \cdot x + b$.

In [ ]:
def lineareF(x,a,b):
    y=a*x+b
    return y

Die Funktion `curve_fit()` erhält als ersten Parameter die definierte Fuktion gefolgt von den `x` und `y` Werten.

Sie gibt zwei Werte zurück. Der erste Rückgabewert enthält die optimierten Parameter, der zweite die Kovarianz als Fehlermass.

In [ ]:
koord,kovar=opt.curve_fit(lineareF,data[:,0],data[:,1])

Wir extrahieren die Koordinaten`a` und `b` und verwenden diese, um die durch die berechnete Funktion bestimmten Funktionswerte `fy` zu berechnen. 

In [ ]:
a,b=koord
fy=lineareF(data[:,0],a,b)

Wir können nun einen Plot erstellen:

In [ ]:
plt.plot(data[:,0],data[:,1],'.', label='original')
plt.plot(data[:,0],fy,'-',label='f(x)')
plt.xlabel('x Werte')
plt.ylabel('y Werte')
plt.legend()

Als Fehlermass berechnen wir das **Bestimmheitsmass $R^2$** und geben es aus. Das Bestimmtheitsmass gibt an, wieviel Prozent des $y$ Wertes durch die $x$-Werte erklärt werden können.

In [ ]:
r2=masse.r2_score(data[:,1],fy)
print(r2)

Ebenso geben wir die Funktion mit den berechneten Koeffizienten aus:

In [ ]:
print('f(x)={:.4f}*x+{:.4f}'.format(a,b))

## 1.3 `polyfit()`

Nebem `curve_fit()` können wir auch die Funktion `polyfit()` verwenden. Die Funktion `polyfit()` ist im Modul `numpy.polynomial.polynomial`.  Sie erwartet drei Parameter: die `x` Werte, die `y` Werte sowie eine Angabe über den Grad des Polynoms. Dieser wird im einfachsten Fall durch eine ganze Zahl angegeben.
Die Methode hat auch weitere optionale Parameter. Sie gibt ein Array mit den aufsteigend geordneten Koeffizienten zurück.

In [ ]:
koeff=poly.polyfit(data[:,0],data[:,1],1)

Die Funktion `polyval()` im selben Modul kann die `y` Werte basierend auf dem Array der aufsteigend sortierten Koeffizienten berechnen. 

In [ ]:
fy=poly.polyval(data[:,0],koeff)

Auch hier können wir wieder einen Plot erstellen:

In [ ]:
plt.plot(data[:,0],data[:,1],'.', label='y')
plt.plot(data[:,0],fy,'-',label='f(x)')
plt.xlabel('x Werte')
plt.ylabel('y Werte')
plt.legend()

Und $R^2$ und die Funktion ausgeben:

In [ ]:
r2=masse.r2_score(data[:,1],fy)
print(r2)

In [ ]:
print('f(x)={:.4f}*x+{:.4f}'.format(a,b))

# 2. Curve-fitting der nicht-linearen Funktion

## 2.1 Kennenlernen der Funktion

Als zweites Beispiel betrachten wir verrauschte Daten der Funktion $f(x)=3.5 \cdot x^3 - 3 \cdot x + 12$, die wir mit der Funktion `kubikDaten()` wie folgt erzeugen:

In [ ]:
data=kubikDaten(a=3.5,b=-3,c=12,div=10,xmin=-5,xmax=10)

Wie zuvor plotten wir als erstes die Funktion:

In [ ]:
plt.plot(data[:,0],data[:,1],'.')

Wenn wir den Pearson's Product Moment Coefficient berechnen, 

In [ ]:
pc,p=stats.pearsonr(data[:,0],data[:,1])
print("Der Korrelationskoeffizient ist {:.4f}.".format(pc))

ist dieser weniger stark ausgeprägt. Wir erinnern uns: der Koeffizient eine **lineare Abhängigkeit** an. 

## 2.2 Curve-Fitting mit einer linearen Funktion

Wie zuvor können wir eine lineare Korrelation annehmen und das Curve Fitting für die Funktion $f(x)=a \cdot x +b$, bzw. ein Polynom ersten Grades berechnen und das Ergebnis plotten. Wir verwenden `polyfit()`: 

In [ ]:
koeff=poly.polyfit(data[:,0],data[:,1],1)
fy=poly.polyval(data[:,0],koeff)
plt.plot(data[:,0],data[:,1],'.', label='y')
plt.plot(data[:,0],fy,'-',label='f(x)')
plt.xlabel('x Werte')
plt.ylabel('y Werte')
plt.legend()

Sowie das Bestimmungsmass berechnen:

In [ ]:
r2=masse.r2_score(data[:,1],fy)
print(r2)

## 2.3 Curve-fitting mit einem Polynom dritten Grades

Wenn wir erkennen, dass die Daten einem Polynom dritten Grades entsprechen, sollten wir das Curve-fitting für ein solches Polynom machen. Am einfachsten verwenden wir die Funktion `polyfit()` und geben 3 für den Grad an:

In [ ]:
koeff=poly.polyfit(data[:,0],data[:,1],3)
fy_1=poly.polyval(data[:,0],koeff)
plt.plot(data[:,0],data[:,1],'.', label='y')
plt.plot(data[:,0],fy_1,'-',label='f(x)')
plt.xlabel('x Werte')
plt.ylabel('y Werte')
plt.legend()

In [ ]:
r2_1=masse.r2_score(data[:,1],fy_1)
print('r2:',r2_1)
f_1='f(x)={:.3f}*x**3+{:.3f}*x**2+{:.3f}*x+{:.3f}'.format(koeff[3],koeff[2],koeff[1],koeff[0])
print(f_1)

Wie Sie feststellen können, berechnet `polyfit()`  die Koeffizienten für alle vier Terme des Polynoms. Wenn wir wissen, dass das Polynom vom Typ $a \cdot x^3+b \cdot x + c$ ist, kann es sinnvoll sein, nur diese Koeffizienten zu berechnen. 
Mit `curve_fit()` können wir dies tun, indem wir die Funktion entsprechend definieren:

In [ ]:
def fKubik(x,a,b,c):
    y=a*x**3+b*x+c
    return y

Und diese dann verwenden:

In [ ]:
popt,pcov=opt.curve_fit(fKubik,data[:,0],data[:,1])
a,b,c=popt
fy_2=fKubik(data[:,0],a,b,c)
plt.plot(data[:,0],data[:,1],'.', label='y')
plt.plot(data[:,0],fy_2,'-',label='f(x)')
plt.xlabel('x Werte')
plt.ylabel('y Werte')
plt.legend()
f_2='f(x)={:.3f}*x**3+{:.3f}*x+{:.3f}'.format(a,b,c)
r2_2=masse.r2_score(data[:,1],fy_2)

Aber auch bei der Verwendung von `polyfit()` können wir einschränken, welche Terme verwendet werden sollen. Anstelle des "einfachen" Grades übergeben wir eine Liste mit dem Grad aller Terme, die beachtet werden sollen. Die Koeffizienten der anderen Terme werden dann zu 0 gesetzt:

In [ ]:
koeff=poly.polyfit(data[:,0],data[:,1],[3,1,0])
fy_3=poly.polyval(data[:,0],koeff)
plt.plot(data[:,0],data[:,1],'.', label='y')
plt.plot(data[:,0],fy_3,'-',label='f(x)')
plt.xlabel('x Werte')
plt.ylabel('y Werte')
plt.legend()
f_3='f(x)={:.3f}*x**3+{:.3f}*x**2+{:.3f}*x+{:.3f}'.format(koeff[3],koeff[2],koeff[1],koeff[0])
r2_3=masse.r2_score(data[:,1],fy_3)

Wir können nun die Ergebnisse vergleichen:

In [ ]:
print('A: polyfit() mit allen vier Termen:' )
print('r2: {}, Funktion: {}'.format(r2_1,f_1))
print('B: polyfit() mit nur drei Termen:' )
print('r2: {}, Funktion: {}'.format(r2_2,f_2))
print('C: curve_fit() mit spezifischer Funktion' )
print('r2: {}, Funktion: {}'.format(r2_3,f_3))

**Hinweis:** Je nachdem, wie Ihre Daten aussehen, ist möglich, dass das Maß in A leicht besser ist. Das liegt daran, dass unsere Daten verrauscht sind, also nicht wirklich den Funktionen entsprechen. Hier können mehr Terme ein genaueres Abbild der Daten ermöglichen. Allerdings bezieht sich das nur auf die Daten, die in das Curve Fitting eingehen. Die zugrundeliegende Funktion wird dadurch nicht besser gefunden. 

Als Beispiel hier noch das Fitting mit einem Polynom 5. Grades:

In [ ]:
koeff=poly.polyfit(data[:,0],data[:,1],5)
fy_4=poly.polyval(data[:,0],koeff)
plt.plot(data[:,0],data[:,1],'.', label='y')
plt.plot(data[:,0],fy_4,'-',label='f(x)')
plt.xlabel('x Werte')
plt.ylabel('y Werte')
plt.legend()
f_4='f(x)={:.3f}*x**5+{:.3f}*x**4+{:.3f}*x**3+{:.3f}*x**2+{:.3f}*x+{:.3f}'.format(koeff[5],koeff[4],koeff[3],koeff[2],koeff[1],koeff[0])
r2_4=masse.r2_score(data[:,1],fy_4)

In [ ]:
print('C: curve_fit() mit spezifischer Funktion' )
print('r2: {}, Funktion: {}'.format(r2_4,f_4))

**Abschlussbemerkung 1:**
Sie haben in diesem Notebook gelernt, wie Sie Curve-Fitting anwenden. Dieses kann einfach durch Aufruf von Funktionen durchgeführt werden. Das Plotten der Kurve und die Angabe der Funktion sehen sehr gut und professionell aus! Aber es ist nicht immer sinnvoll! Die Funktionen bilden eine gegebene Menge von Werten möglichst gut ab. Daraus Schlussfolgerungen auf andere Werte und Zusammenhänge zu ziehen, ist in manchen Fällen sinnvoll, manchesmal auch nicht. Vergessen Sie daher nie, Ergebnisse kritisch zu betrachten und sich selbst und anderen Fragen zu stellen!

**Abschlussbemerkung 2:**
Sowohl `curve_fit()` als auch `polyfit()` verwenden die Methode der kleinsten Quadrate für die lineare Regression. Lineare Regression ist eine Methode des maschinellen Lernens. Mehr Terme einzubeziehen, also z.B. den Grad zu erhöhen, kann zu einer besseren Anpassung der Funktion an **die gegebenen Daten** führen, erhöht aber zugleich das Risiko einer Überanpassung. Diese sollte vermieden werden, wenn man die berechnete Funktion zur Vorhersage von Werten nutzen möchte. 

*Ende des Notebooks*

<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Dieses Notebook wurde von Christina B. Class für die Lehre an der EAH Jena erstellt. Es ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">Creative Commons Namensnennung - Nicht kommerziell - Keine Bearbeitungen 4.0 International Lizenz</a>.